In [1]:
import numpy as np
import pandas as pd
import healpy as hp
import os
from DS_Planck_Unet import *
from DS_healpix_fragmentation import radec2pix
from matplotlib import pyplot as plt
from DS_data_transformation import draw_df
from DS_detector import *
from tqdm.notebook import tqdm
import re

In [4]:
models = {10 : load_planck_model('/home/rt2122/Models/act_cut/act_cut.ep0010-vl0.034219-l0.032022.hdf5'),
    14 : load_planck_model('/home/rt2122/Models/act_cut/act_cut.ep0014-vl0.035575-l0.029666.hdf5')}

In [5]:
thr = 0.1

In [7]:
mod_dict = {10 : [], 14 : []}
for ep in models:
    for i in tqdm(range(48)):
        all_dict = gen_pics_for_detection(i, models[ep], step=8)
        coords = detect_clusters_connected(all_dict, thr, i)
        mod_dict[ep].append(coords)
    mod_dict[ep] = pd.concat(mod_dict[ep], ignore_index=True)
    mod_dict[ep].to_csv(
        '/home/rt2122/Data/scans_extended/connected/full_act_cut_ep{ep}_thr{thr}_step8.csv'.format(
            ep=ep, thr=0.1), index=False)

In [8]:
act_cut10 = pd.read_csv('/home/rt2122/Data/scans_extended/connected/full_act_cut_ep10_thr0.1_step8.csv')
act_cut14 = pd.read_csv('/home/rt2122/Data/scans_extended/connected/full_act_cut_ep14_thr0.1_step8.csv')

In [12]:
def recall(df, cat):
    import numpy as np
    if cat == 'fp':
        return np.count_nonzero(df['status'] == 'fp')
    df = df[df['catalog'] == cat]
    return np.count_nonzero(df['status'] == 'tp') / len(df)

In [22]:
stat_df = pd.DataFrame({cat : [recall(act_cut10, cat), recall(act_cut14, cat)] for 
                       cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'act_erosita', 'fp']},
                      index=[10, 14])

In [23]:
stat_df

,planck_z,planck_no_z,mcxcwp,act_erosita,fp
10,0.905850,0.651163,0.132439,0.179195,16316
14,0.909506,0.644007,0.140821,0.292282,16484
